# Overview

This example shows how to extract a time series of images from Earth Engine, annotate them, and save them as a video.

# Initialize Earth Engine

We start by importing the [Earth Engine Python API](https://pypi.org/project/earthengine-api/) module.

In [ ]:
import ee

The following command initializes the Earth Engine Python API.

In [ ]:
ee.Initialize()

If the cell produces output that displays an error about needing to authenticate, open up the notebook entitled `01 - Setup auth credentials` and follow the instructions.

# Define Helper Functions

In [ ]:
def GetTileLayerUrl(ee_image_object):
    map_id = ee.Image(ee_image_object).getMapId()
    tile_url_template = "https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}"
    return tile_url_template.format(**map_id)

# Building an image collection

In [ ]:
import datetime
import ipywidgets as widgets
import ipyleaflet  # an interactive mapping "widget"
from sidecar import Sidecar

## Define the Image Collection to work with

In [ ]:
vis_params = {
    'bands': ['HV'],
    'min':-20,
    'max':0,
}

## Define User Interface Elements

In [ ]:
roi_dimension = widgets.IntSlider(
    value=1e4,
    min=1e2,
    max=2e4,
    description='ROI Size (m):',
    continuous_update=False,
)

In [ ]:
# Define the map.
map1 = ipyleaflet.Map(
    center=(81.736, -45.912),
    zoom=8,
    layout={'height':'300px'},
)
map1.add_control(ipyleaflet.LayersControl())

# Define and add a Marker pin to the map.
center_marker = ipyleaflet.Marker(
    name='ROI Selection Marker',
    location=map1.center
)
map1 += center_marker

mosaic_layer_group = ipyleaflet.LayerGroup(layers=(), name='Mosaic Layer')
map1 += mosaic_layer_group

roi_layer_group = ipyleaflet.LayerGroup(layers=(), name='ROI Layer')
map1 += roi_layer_group

In [ ]:
start_datepicker = widgets.DatePicker(
    description='Start Date',
    disabled=False,
    value=datetime.datetime(2017, 8, 1)
)
end_datepicker = widgets.DatePicker(
    description='End Date',
    disabled=False,
    value=datetime.datetime(2017, 8, 10)
)

In [ ]:
out = widgets.Output()

## Display the UI Elements

In [ ]:
# Layout the UI elements.
panel = widgets.VBox([
    map1,
    start_datepicker,
    end_datepicker,
    roi_dimension,
    out
])

out.clear_output()
display(panel)

## Define interactions

In [ ]:
# Define helper functions to swap the coordinate ordering.
def swap_coordinate_xy_for_location(coord):
    return (coord[1],coord[0])

def swap_coordinate_xy_for_list(coord_list):
    return [swap_coordinate_xy_for_location(coord) for coord in coord_list]  

def update_roi_layer(map_reference):    
    coord_list_xy = get_roi_polygon()['coordinates'][0]
    coord_list_yx = swap_coordinate_xy_for_list(coord_list_xy)
    
    roi_layer = ipyleaflet.Polygon(
        name='TEST update ROI Polygon',
        locations=coord_list_yx,
        weight=3,
        color='#F00',
        opacity=0.8,
        fill_opacity=0.1,
        fill_color='#F00'
    )
    roi_layer_group.clear_layers()
    roi_layer_group.add_layer(roi_layer)

In [ ]:
def get_roi_polygon():
    center_marker_xy = swap_coordinate_xy_for_location(center_marker.location)
    centroid = ee.Geometry.Point(center_marker_xy)
    buffered = centroid.buffer(roi_dimension.value).bounds()
    return buffered.getInfo() 

In [ ]:
def get_image_collection():

    # Get filter values from the UI widgets.
    roi = get_roi_polygon()
    start_date = ee.Date(start_datepicker.value.isoformat())
    end_date = ee.Date(end_datepicker.value.isoformat())

    collection = (
        base_collection
          .filterDate(start_date, end_date)
          .filterBounds(roi)
    )
    return collection

In [ ]:
base_collection = (
    ee.ImageCollection('COPERNICUS/S1_GRD')
      .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HV'))
      .select(['HV'])
)

In [ ]:
def update_mosaic_layer(map_reference):
    
    out.clear_output()
    with out:
        print('Total images = {0}'.format(get_image_collection().size().getInfo()))
    
    mosaic_tilelayer = ipyleaflet.TileLayer(
        url=GetTileLayerUrl(
            get_image_collection().mosaic().visualize(**vis_params)
        ),
        attribution='Map tiles by <a href="http://earthengine.google.com/">Earth Engine</a>.'
    )
    mosaic_layer_group.clear_layers()
    mosaic_layer_group.add_layer(mosaic_tilelayer)

In [ ]:
# Define the actions performed when the marker moves.
def center_marker_on_move(change):
    update_roi_layer(map1)
    update_mosaic_layer(map1)
center_marker.unobserve_all()
center_marker.observe(center_marker_on_move, names='location')

In [ ]:
# Define the actions performed when the ROI size is changed.
def roi_dimension_on_change(change):
    update_roi_layer(map1)
    update_mosaic_layer(map1)
roi_dimension.unobserve_all()
roi_dimension.observe(roi_dimension_on_change, names='value')

Initialize the overlay layers.

In [ ]:
update_roi_layer(map1)
update_mosaic_layer(map1)

# Testing out a series

This section will demonstrate outputing an image time series for the specified location, time interval, and image collection.

In [ ]:
collection = get_image_collection()
print(collection.size().getInfo())

In [ ]:
MAX_ELEMENTS=100
images = collection.toList(MAX_ELEMENTS).getInfo()

In [ ]:
[img['id'] for img in images]

In [ ]:
import os
output_directory = 'output'
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

In [ ]:
import datetime
from IPython.display import Image
import imageio
import pprint
import PIL.Image
import PIL.ImageDraw
import PIL.ImageFont
import requests
import numpy

pp = pprint.PrettyPrinter(indent=4)

image_no = 0

for img in images:
    image_no += 1
    id = img['id']
    print(id)

    image_time_ms = img['properties']['system:time_start']
    start_time = datetime.datetime.fromtimestamp(image_time_ms/1000.0)

    sample = ee.Image(id)
    url = sample.getThumbUrl({
        'bands': 'HH',
        'min':-20,
        'max':0,
        'region':get_roi_polygon()
    })

    # Create a base image, from the Earth Engine Thumbnail URL.
    base_ee = PIL.Image.open(requests.get(url, stream=True).raw).convert('RGBA')

    # Create a background image.
    background = PIL.Image.new('RGBA', base_ee.size, (12,12,12,255))

    # Burn the EE data onto the backgound.
    base = PIL.Image.alpha_composite(background, base_ee)

    # make a blank image for the text, initialized to transparent text color
    txt = PIL.Image.new('RGBA', base_ee.size, (255,255,255,0))

    # Get a font.
    # Additional fonts available on this server can be found by running the following in a terminal:
    # find / -name *.ttf
    fnt = PIL.ImageFont.truetype('/usr/share/fonts/truetype/liberation/LiberationMono-Bold.ttf', 30)

    # Get a drawing context.
    d = PIL.ImageDraw.Draw(txt)
    # Draw text. For date formatting codes see:
    # https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior
    date_string = '{:%Y-%m-%d %H:%M:%S}'.format(start_time)
    d.text((10,10), date_string, font=fnt, fill=(255,0,0,255))
    # Composite the image and text annotation.
    out_image = PIL.Image.alpha_composite(base, txt)

    outfile = '{0}/img{1:03d}.png'.format(output_directory, image_no)
    out_image.save(outfile, format='PNG')

    display(out_image)

## Create an animated GIF.

In [ ]:
import glob

gif_images = []
filenames = sorted(glob.glob('output/img???.png'))
for filename in filenames:
    gif_images.append(imageio.imread(filename))
imageio.mimsave('output/movie.gif', gif_images, 'GIF', duration=0.5)

## Create an MPEG video.

In [ ]:
%%bash  
ffmpeg -r 1 \
    -i output/img%03d.png \
    -vcodec mpeg4 \
    -y output/out.mp4